In [2]:
import psycopg2

from src.config.configurations import ConfigurationManager
from src.data_connectors.config_database import RDSConnector, RedshiftConnector

config = ConfigurationManager()

rds_config = config.get_rds_config()
rds_con = RDSConnector(rds_config)
# rds_con.get_tables('decision')
print(rds_con.get_dataframe("select id from decision.loan limit 5"))


2024-12-23 03:26:53.370 | INFO     | src.utils.common:read_yaml:27 - yaml file: config/config.yaml loaded successfully


   id
0   1
1   2
2   3
3   4
4   5


In [2]:
config = ConfigurationManager()
redshift_config = config.get_redshift_config()
redshift_con = RedshiftConnector(redshift_config)
redshift_con.get_databases()
redshift_con.get_dataframe("select lead_id from adf0275.poold_analysis_20240628 limit 5", "analytics")

2024-12-20 07:20:45.006 | INFO     | src.utils.common:read_yaml:27 - yaml file: config/config.yaml loaded successfully
/home/ubuntu/end_to_end_ml/src/database/config_database.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
2024-12-20 07:20:45.033 | INFO     | src.database.config_database:get_databases:123 - No of Databases: 22


,lead_id
0,309478
1,299472
2,309737
3,322638
4,327202


In [1]:
import os
import polars as pl
import time
import pandas as pd
# os.environ["AWS_REGION"] = "your-region"
idvs = ['tu_income_estimator', 'misaligned_pay_freqency', 'improbable_pay_date', 'infrequent_pay_date', 'paymentstartdate_paydate1_diff', 'paydate1_application_create_date_diff', 'paydate1_lead_create_date_diff', 'paymentstartdate_paydate1_same_flag', 'paydate1_day', 'paydate1_weekday', 'paymentstartdate_weekday', 'lead_create_date_weekday', 'application_create_date_weekday', 'paymentstartdate_paydate1_notsame_and_weekend_flag', 'paydate1_holiday_flag', 'paydate1_holiday_or_weekend_flag', 'paymentstartdate_holiday_flag', 'paymentstartdate_holiday_or_weekend_flag', 'goverment_benefits_paydate_alignment_flag', 'ft_emp_logic_employerdomainmatches', 'ft_emp_logic_employermatchpercentage', 'ft_emp_logic_employermatchratio', 'ft_emp_logic_employernames', 'ft_emp_logic_homeziptoworkzipdistances', 'ft_emp_logic_ipaddressorigindomains', 'ft_emp_logic_iporigintoworkzipdistances', 'ft_emp_logic_monthlyincomes', 'ft_emp_logic_paydatematchcount', 'ft_emp_logic_paydatematchpercentage', 'ft_emp_logic_paydatematchprojected', 'ft_emp_logic_paydatematchratio', 'ft_emp_logic_payfrequencies', 'ft_emp_logic_payfrequencymatchpercentage', 'ft_emp_logic_payfrequencymatchratio', 'ft_emp_logic_payrolltypes', 'ft_emp_logic_projectedpaydate', 'ft_emp_logic_score', 'ft_emp_logic_scorepercentage', 'ft_emp_logic_workzips', 'emp1_first_match_ratio', 'emp1_is_abbr', 'emp1_is_any_match', 'emp1_last_match_ratio', 'emp1_match_ratio', 'emp_first_match_ratio', 'emp_is_abbr', 'emp_is_any_match', 'emp_last_match_ratio', 'emp_match_ratio', 'emp_max_first_match_ratio', 'emp_max_last_match_ratio', 'emp_max_match_ratio', 'employment_1_employer', 'employment_1_employer_unparsed', 'employment_employer', 'employment_employer_unparsed', 'no_of_times_job_switched_12_months', 'no_of_times_job_switched_1_months', 'no_of_times_job_switched_24_months', 'no_of_times_job_switched_2_months', 'no_of_times_job_switched_36_months', 'no_of_times_job_switched_3_months', 'no_of_times_job_switched_48_months', 'no_of_times_job_switched_60_months', 'no_of_times_job_switched_6_months', 'no_of_times_job_switched_96_months', 'no_of_times_job_switched_ever', 'lti', 'payment_freq_bi_weekly', 'payment_freq_monthly', 'payment_freq_twice_per_month', 'payment_freq_weekly', 'payment_freq_other', 'repayment_freq_bi_weekly', 'repayment_freq_monthly', 'repayment_freq_twice_per_month', 'repayment_freq_weekly', 'employmentstatus_employment', 'employmentstatus_selfemployed', 'employmentstatus_governmentbenefit', 'employmentstatus_pension_or_annuity', 'employmentstatus_other', 'pti_1', 'pti_2', 'cca_num_of_employers_last_six_months', 'cca_reported_net_monthly_income_previously_seen', 'RTI', 'DTI_current_debt', 'DTI_current_usi_debt', 'stated_income', 'income_rounding']

In [3]:
pd.read_parquet('s3://ml-framework-prod/dheerajks/employment_analyses/data/train.parquet')

/tmp/ipykernel_10149/3645981205.py:1: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  pl.read_parquet('s3://ml-framework-prod/dheerajks/employment_analyses/data/train.parquet')


tu_income_estimator,misaligned_pay_freqency,improbable_pay_date,infrequent_pay_date,paymentstartdate_paydate1_diff,paydate1_application_create_date_diff,paydate1_lead_create_date_diff,paymentstartdate_paydate1_same_flag,paydate1_day,paydate1_weekday,paymentstartdate_weekday,lead_create_date_weekday,application_create_date_weekday,paymentstartdate_paydate1_notsame_and_weekend_flag,paydate1_holiday_flag,paydate1_holiday_or_weekend_flag,paymentstartdate_holiday_flag,paymentstartdate_holiday_or_weekend_flag,goverment_benefits_paydate_alignment_flag,ft_emp_logic_employerdomainmatches,ft_emp_logic_employermatchpercentage,ft_emp_logic_employermatchratio,ft_emp_logic_employernames,ft_emp_logic_homeziptoworkzipdistances,ft_emp_logic_ipaddressorigindomains,ft_emp_logic_iporigintoworkzipdistances,ft_emp_logic_monthlyincomes,ft_emp_logic_paydatematchcount,ft_emp_logic_paydatematchpercentage,ft_emp_logic_paydatematchprojected,ft_emp_logic_paydatematchratio,ft_emp_logic_payfrequencies,ft_emp_logic_payfrequencymatchpercentage,ft_emp_logic_payfrequencymatchratio,ft_emp_logic_payrolltypes,ft_emp_logic_projectedpaydate,ft_emp_logic_score,…,payment_freq_monthly,payment_freq_twice_per_month,payment_freq_weekly,payment_freq_other,repayment_freq_bi_weekly,repayment_freq_monthly,repayment_freq_twice_per_month,repayment_freq_weekly,employmentstatus_employment,employmentstatus_selfemployed,employmentstatus_governmentbenefit,employmentstatus_pension_or_annuity,employmentstatus_other,pti_1,pti_2,cca_num_of_employers_last_six_months,cca_reported_net_monthly_income_previously_seen,RTI,DTI_current_debt,DTI_current_usi_debt,stated_income,income_rounding,employer_mean_salary,employer_mean_std,employer_salary_standardized,employer_median_salary,payment_freq_percentage,inc_by_avginc_mapped_occ,inc_by_avginc_low_summ_occ,inc_by_avginc_high_summ_occ,inc_by_blsinc_mapped_occ,inc_by_blsinc_low_summ_occ,inc_by_blsinc_high_summ_occ,inc_by_state_wise_avginc,inc_by_city_wise_avginc,_6pd30,lead_id
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,i64
0.0,-0.476128,-0.148182,0.55397,-0.885678,-0.232508,-0.230703,-0.488358,-1.523284,-2.459607,-0.975085,-0.802303,-0.810676,-0.290787,-0.084695,-0.307839,-0.098522,-0.363207,-0.230211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.19659,-0.168825,-0.171862,-0.03859,-0.446951,-0.209992,-0.166585,0.0,-2.142357,-0.269134,-0.215371,4.390487,-0.134284,1.870176,0.510104,0.0,0.0,-0.658725,0.0,0.0,-0.090994,-1.995103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.08797,0.0,0.0,42
0.0,2.100275,-0.148182,0.55397,-0.885678,-0.232508,-0.230703,0.201376,-0.04877,0.484039,-0.975085,1.349268,1.340473,-0.290787,-0.084695,-0.307839,-0.098522,-0.363207,-0.230211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.19659,-0.168825,-0.171862,-0.03859,-0.446951,-0.209992,-0.166585,0.0,0.466776,-0.269134,-0.215371,-0.227765,-0.134284,1.404958,0.215265,0.0,0.0,0.764018,0.0,0.0,-0.101112,0.501227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.09869,-0.08512,1.0,93
0.0,-0.476128,-0.148182,-1.805153,-0.885678,-0.232508,-0.230703,-0.419384,0.79381,1.21995,-1.67975,-1.340195,-1.348464,-0.290787,-0.084695,3.24845,-0.098522,-0.363207,-0.230211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,-0.19659,-0.168825,-0.171862,-0.03859,-0.446951,-0.209992,-0.166585,0.0,0.466776,-0.269134,-0.215371,-0.227765,-0.134284,-0.988233,-1.034137,0.0,0.0,-0.401959,0.0,0.0,-0.056389,-1.995103,-0.299425,-0.322842,0.900945,0.450122,0.790207,0.0,0.0,0.0,0.0,0.0,0.0,-0.051308,0.0,0.0,127
0.0,-0.476128,-0.148182,0.55397,-0.885678,-0.232508,-0.230703,0.408297,0.688487,0.484039,1.138908,-0.802303,-0.810676,3.438948,-0.084695,-0.307839,-0.098522,2.753249,-0.23021

In [110]:
file_path = 's3://ml-framework-prod/dheerajks/employment_analyses/data/income_model_idvs_2024_10_03.parquet'

# Load using Polars
start = time.time()
df = pl.read_parquet(file_path)
end = time.time()
print(f"Polars Load Time: {end - start:.2f} seconds")

Polars Load Time: 1.31 seconds


In [111]:
numerical_cols = [col for col, dtype in zip(df.columns,df.dtypes) if dtype in (pl.Int64,  pl.Int32, pl.Float64, pl.Float32)]
datetime_cols = [col for col, dtype in zip(df.columns,df.dtypes) if dtype == pl.Datetime]
object_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype == pl.Utf8]
object_cols

['loanid',
 'leadid',
 'starter_loan_flag',
 'channel',
 'paymentfrequency',
 'repaymentfrequency']

In [112]:
df = df.with_columns(
    pl.col('starter_loan_flag').str.replace('true', True).str.replace('false', False).cast(pl.Int32).alias('starter_loan_flag')
)

In [106]:
from sklearn.preprocessing import StandardScaler

In [113]:

df = df.with_columns(
[pl.col(numerical_cols).fill_null(float('nan'))]
)
df = df.with_columns([
    pl.col(numerical_cols).replace([float('inf'), float('-inf')], None)
])

In [108]:
df.select(numerical_cols)

lead_id,employmentstatus,stated_income,requestamount,repaymentmethod,tu_income_estimator,misaligned_pay_freqency,improbable_pay_date,infrequent_pay_date,due_dt_disbursal_date_diff,paymentstartdate_disbursal_date_diff,due_dt_paydate1_diff,paymentstartdate_paydate1_diff,paydate1_application_create_date_diff,paydate1_lead_create_date_diff,paymentstartdate_paydate1_same_flag,due_dt_paydate1_same_flag,paydate1_day,paydate1_weekday,paymentstartdate_weekday,lead_create_date_weekday,application_create_date_weekday,due_dt_weekday,paymentstartdate_paydate1_notsame_and_weekend_flag,due_dt_paydate1_notsame_and_weekend_flag,paydate1_holiday_flag,paydate1_holiday_or_weekend_flag,paymentstartdate_holiday_flag,paymentstartdate_holiday_or_weekend_flag,due_dt_holiday_flag,due_dt_holiday_or_weekend_flag,goverment_benefits_paydate_alignment_flag,ft_emp_logic_employerdomainmatches,ft_emp_logic_employermatchpercentage,ft_emp_logic_employermatchratio,ft_emp_logic_employernames,ft_emp_logic_homeziptoworkzipdistances,…,employment_employer_unparsed,no_of_times_job_switched_12_months,no_of_times_job_switched_1_months,no_of_times_job_switched_24_months,no_of_times_job_switched_2_months,no_of_times_job_switched_36_months,no_of_times_job_switched_3_months,no_of_times_job_switched_48_months,no_of_times_job_switched_60_months,no_of_times_job_switched_6_months,no_of_times_job_switched_96_months,no_of_times_job_switched_ever,lti,income_rounding,payment_freq_bi_weekly,payment_freq_monthly,payment_freq_twice_per_month,payment_freq_weekly,payment_freq_other,repayment_freq_bi_weekly,repayment_freq_monthly,repayment_freq_twice_per_month,repayment_freq_weekly,employmentstatus_employment,employmentstatus_selfemployed,employmentstatus_governmentbenefit,employmentstatus_pension_or_annuity,employmentstatus_other,pti_1,pti_2,current_debt,current_usi_debt,cca_num_of_employers_last_six_months,cca_reported_net_monthly_income_previously_seen,RTI,DTI_current_debt,DTI_current_usi_debt
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.992233e6,1.0,42000.0,2000.0,1.0,NaN,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.0,15.0,3.0,4.0,1.0,2.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,0.927059,NaN,NaN,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.025,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.029109,0.058217,787.0,NaN,1.0,5000.0,0.047619,0.018738,NaN
2.636831e6,1.0,54600.0,3000.0,4.0,NaN,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,30.0,2.0,2.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.028846,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.031782,0.068862,851.0,690.0,2.0,4500.0,0.054945,0.015586,0.012637
4.471053e6,1.0,26400.0,4000.0,1.0,39999.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,5.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.563348,NaN,NaN,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.059659,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0506,0.109633,308.0,NaN,0.0,2600.0,0.151515,0.011667,NaN
5.127363e6,1.0,39600.0,3000.0,1.0,39999.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,18.0,4.0,4.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.242857,NaN,NaN,…,NaN,1.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,0.025253,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.025191,0.05458,146.0,42.0,2.0,4000.0,0.075758,0.003687,0.001061
5.453198e6,7.0,28080.0,1000.0,1.0,39999.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,21.0,10.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.018697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.010466,0.022676,140.0,101.0,0.0

In [109]:
scl = StandardScaler()
scl.fit(df.select(numerical_cols).to_numpy())

scalred_data = scl.transform(df.select(numerical_cols).to_numpy())
scalred_df = pl.DataFrame(scalred_data, schema=numerical_cols)
df.with_columns([scalred_df[col] for col in numerical_cols])

/home/ubuntu/anaconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/ubuntu/anaconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/home/ubuntu/anaconda3/lib/python3.10/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


loanid,lead_id,leadid,disbursal_date,due_dt,payment_dt,starter_loan_flag,channel,paydate1,lead_create_date,employmentstatus,stated_income,requestamount,paymentfrequency,repaymentfrequency,paymentstartdate,application_create_date,repaymentmethod,tu_income_estimator,misaligned_pay_freqency,improbable_pay_date,infrequent_pay_date,due_dt_disbursal_date_diff,paymentstartdate_disbursal_date_diff,due_dt_paydate1_diff,paymentstartdate_paydate1_diff,paydate1_application_create_date_diff,paydate1_lead_create_date_diff,paymentstartdate_paydate1_same_flag,due_dt_paydate1_same_flag,paydate1_day,paydate1_weekday,paymentstartdate_weekday,lead_create_date_weekday,application_create_date_weekday,due_dt_weekday,paymentstartdate_paydate1_notsame_and_weekend_flag,…,employment_employer_unparsed,no_of_times_job_switched_12_months,no_of_times_job_switched_1_months,no_of_times_job_switched_24_months,no_of_times_job_switched_2_months,no_of_times_job_switched_36_months,no_of_times_job_switched_3_months,no_of_times_job_switched_48_months,no_of_times_job_switched_60_months,no_of_times_job_switched_6_months,no_of_times_job_switched_96_months,no_of_times_job_switched_ever,lti,income_rounding,payment_freq_bi_weekly,payment_freq_monthly,payment_freq_twice_per_month,payment_freq_weekly,payment_freq_other,repayment_freq_bi_weekly,repayment_freq_monthly,repayment_freq_twice_per_month,repayment_freq_weekly,employmentstatus_employment,employmentstatus_selfemployed,employmentstatus_governmentbenefit,employmentstatus_pension_or_annuity,employmentstatus_other,pti_1,pti_2,current_debt,current_usi_debt,cca_num_of_employers_last_six_months,cca_reported_net_monthly_income_previously_seen,RTI,DTI_current_debt,DTI_current_usi_debt
str,f64,str,datetime[ns],datetime[ns],datetime[ns],i32,str,datetime[ns],datetime[ns],f64,f64,f64,str,str,datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""LAI-00092480""",-1.076019,"""546838288177@19H1""",2019-08-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00,1,"""PS""",2019-08-15 00:00:00,2019-07-30 16:55:37,-0.369538,-0.092636,-0.300962,"""TWICE_PER_MONTH""","""TWICE_PER_MONTH""",2019-08-16 00:00:00,2019-07-31 08:59:13,-0.154608,NaN,-0.473863,-0.143303,-0.551019,0.0,-0.010235,-0.279645,-0.886403,-0.235093,-0.233417,-0.480314,-0.203687,-0.047201,-0.252,0.435386,-0.802518,-0.272932,1.824517,-0.289477,…,NaN,-0.487948,-0.138423,-0.693191,-0.195868,-0.867476,-0.241434,0.294052,0.103961,-0.344924,-0.319042,-2.258978,-0.972317,0.503115,-0.459358,-0.224623,-0.185014,-0.197738,-0.045518,-0.513445,-0.239078,-0.182912,0.0,0.467579,-0.268262,-0.214394,-0.231644,-0.133079,-0.391715,-0.318498,-0.192726,NaN,0.291693,-0.001461,-0.245758,-0.004346,NaN
"""LAI-00121272""",-0.855784,"""576839437287@19K1""",2019-10-25 00:00:00,2019-11-27 00:00:00,null,1,"""PS""",2019-10-30 00:00:00,2019-10-24 14:48:33,-0.369538,-0.076328,-0.080018,"""BI_WEEKLY""","""BI_WEEKLY""",2019-11-13 00:00:00,2019-10-24 14:51:11,7.140217,NaN,-0.473863,-0.143303,1.814819,0.0,-0.010235,-0.279645,-0.886403,-0.235093,-0.233417,0.290216,0.439973,1.533356,-0.988352,-0.974444,0.272472,0.264548,-0.974354,-0.289477,…,NaN,-0.487948,-0.138423,-0.693191,-0.195868,-0.867476,-0.241434,-1.033877,-1.194419,-0.344924,-0.319042,0.442678,-0.903053,0.503115,-0.459358,-0.224623,-0.185014,-0.197738,-0.045518,-0.513445,-0.239078,-0.182912,0.0,0.467579,-0.268262,-0.214394,-0.231644,-0.133079,-0.305217,-0.098889,-0.132194,0.58776,1.29251,-0.032165,-0.182499,-0.215052,0.680975
"""LAI-00201559""",-0.229098,"""699961605241@21C1""",2021-03-02 00:00:00,2021-04-02 00:00:00,2021-04-02 00:00:00,1,"""PS""",2021-03-05 00:00:00,2021-03-01 14:01:00,-0.369538,-0.112826,0.140926,"""BI_WEEKLY""","""BI_WEEKLY""",2021-03-19 00:00:00,2021-03-01 14:05:31,-0.154608,-0.560291,-0.473863,-0.143303,-0.551019,0.0,-0.010235,-0.2796

In [120]:
df.select(pl.col('channel').n_unique())

channel
u32
20


In [121]:
df['channel'].value_counts(normalize=True)["proportion"].max()

0.5083328760768391

In [97]:
class DataCleaning:
    def __init__(self, file_path, categorical_treshold_frequency=0.25):
        self.file_path = file_path
        self.categorical_treshold_frequency = categorical_treshold_frequency

    def load_data(self, path):
        return pl.read_parquet(path)
    
    def clean_data(self):
        sampling = ['train', 'val', 'test'] if test_sampling_present else ['train', 'val']

        self.data = {sample: self.load_data(f'{path}{sample}.parquet') for sample in sampling}

        for sample, df in self.data.items():
            # Exclude datetime columns
            datetime_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype == pl.Datetime]
            df = df.drop(datetime_cols)

            object_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype == pl.Utf8]
            high_cardinality_cols = []
            one_hot_cat_cols = []

            for col in object_cols:
                max_freq = df[col].value_counts(normalize=True)["proportion"].max()
                if max_freq > self.categorical_treshold_frequency:
                    one_hot_cat_cols.append(col)
                else:
                    high_cardinality_cols.append(col)

            df = df.with_columns(
                [pl.col(col).str.strip_chars().alias(col) for col in one_hot_cat_cols]
            )

            df = df.drop(columns=high_cardinality_cols)
        
            
            for col in df.columns:
                try:
                    df = df.with_columns(
                        [pl.col(col).cast(float).alias(col)]
                    )
                except:
                    pass
            
            df = df.with_columns(
                [pl.col(col).cast(pl.Categorical).to_dummies().alias(col) for col in one_hot_cat_cols]
            )
            self.data[sample] = df

        

            


loanid,lead_id,leadid,disbursal_date,due_dt,payment_dt,starter_loan_flag,channel,paydate1,lead_create_date,employmentstatus,stated_income,requestamount,paymentfrequency,repaymentfrequency,paymentstartdate,application_create_date,repaymentmethod,tu_income_estimator,misaligned_pay_freqency,improbable_pay_date,infrequent_pay_date,due_dt_disbursal_date_diff,paymentstartdate_disbursal_date_diff,due_dt_paydate1_diff,paymentstartdate_paydate1_diff,paydate1_application_create_date_diff,paydate1_lead_create_date_diff,paymentstartdate_paydate1_same_flag,due_dt_paydate1_same_flag,paydate1_day,paydate1_weekday,paymentstartdate_weekday,lead_create_date_weekday,application_create_date_weekday,due_dt_weekday,paymentstartdate_paydate1_notsame_and_weekend_flag,…,employment_employer_unparsed,no_of_times_job_switched_12_months,no_of_times_job_switched_1_months,no_of_times_job_switched_24_months,no_of_times_job_switched_2_months,no_of_times_job_switched_36_months,no_of_times_job_switched_3_months,no_of_times_job_switched_48_months,no_of_times_job_switched_60_months,no_of_times_job_switched_6_months,no_of_times_job_switched_96_months,no_of_times_job_switched_ever,lti,income_rounding,payment_freq_bi_weekly,payment_freq_monthly,payment_freq_twice_per_month,payment_freq_weekly,payment_freq_other,repayment_freq_bi_weekly,repayment_freq_monthly,repayment_freq_twice_per_month,repayment_freq_weekly,employmentstatus_employment,employmentstatus_selfemployed,employmentstatus_governmentbenefit,employmentstatus_pension_or_annuity,employmentstatus_other,pti_1,pti_2,current_debt,current_usi_debt,cca_num_of_employers_last_six_months,cca_reported_net_monthly_income_previously_seen,RTI,DTI_current_debt,DTI_current_usi_debt
str,f64,str,datetime[ns],datetime[ns],datetime[ns],i32,str,datetime[ns],datetime[ns],f64,f64,f64,str,str,datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""LAI-00092480""",-1.076019,"""546838288177@19H1""",2019-08-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00,1,"""PS""",2019-08-15 00:00:00,2019-07-30 16:55:37,-0.369538,-0.092636,-0.300962,"""TWICE_PER_MONTH""","""TWICE_PER_MONTH""",2019-08-16 00:00:00,2019-07-31 08:59:13,-0.154608,NaN,-0.473863,-0.143303,-0.551019,0.0,-0.010235,-0.279645,-0.886403,-0.235093,-0.233417,-0.480314,-0.203687,-0.047201,-0.252,0.435386,-0.802518,-0.272932,1.824517,-0.289477,…,NaN,-0.487948,-0.138423,-0.693191,-0.195868,-0.867476,-0.241434,0.294052,0.103961,-0.344924,-0.319042,-2.258978,-0.972317,0.503115,-0.459358,-0.224623,-0.185014,-0.197738,-0.045518,-0.513445,-0.239078,-0.182912,0.0,0.467579,-0.268262,-0.214394,-0.231644,-0.133079,-0.391715,-0.318498,-0.192726,NaN,0.291693,-0.001461,-0.245758,-0.004346,NaN
"""LAI-00121272""",-0.855784,"""576839437287@19K1""",2019-10-25 00:00:00,2019-11-27 00:00:00,null,1,"""PS""",2019-10-30 00:00:00,2019-10-24 14:48:33,-0.369538,-0.076328,-0.080018,"""BI_WEEKLY""","""BI_WEEKLY""",2019-11-13 00:00:00,2019-10-24 14:51:11,7.140217,NaN,-0.473863,-0.143303,1.814819,0.0,-0.010235,-0.279645,-0.886403,-0.235093,-0.233417,0.290216,0.439973,1.533356,-0.988352,-0.974444,0.272472,0.264548,-0.974354,-0.289477,…,NaN,-0.487948,-0.138423,-0.693191,-0.195868,-0.867476,-0.241434,-1.033877,-1.194419,-0.344924,-0.319042,0.442678,-0.903053,0.503115,-0.459358,-0.224623,-0.185014,-0.197738,-0.045518,-0.513445,-0.239078,-0.182912,0.0,0.467579,-0.268262,-0.214394,-0.231644,-0.133079,-0.305217,-0.098889,-0.132194,0.58776,1.29251,-0.032165,-0.182499,-0.215052,0.680975
"""LAI-00201559""",-0.229098,"""699961605241@21C1""",2021-03-02 00:00:00,2021-04-02 00:00:00,2021-04-02 00:00:00,1,"""PS""",2021-03-05 00:00:00,2021-03-01 14:01:00,-0.369538,-0.112826,0.140926,"""BI_WEEKLY""","""BI_WEEKLY""",2021-03-19 00:00:00,2021-03-01 14:05:31,-0.154608,-0.560291,-0.473863,-0.143303,-0.551019,0.0,-0.010235,-0.2796

In [86]:
df

loanid,lead_id,leadid,disbursal_date,due_dt,payment_dt,starter_loan_flag,channel,paydate1,lead_create_date,employmentstatus,stated_income,requestamount,paymentfrequency,repaymentfrequency,paymentstartdate,application_create_date,repaymentmethod,tu_income_estimator,misaligned_pay_freqency,improbable_pay_date,infrequent_pay_date,due_dt_disbursal_date_diff,paymentstartdate_disbursal_date_diff,due_dt_paydate1_diff,paymentstartdate_paydate1_diff,paydate1_application_create_date_diff,paydate1_lead_create_date_diff,paymentstartdate_paydate1_same_flag,due_dt_paydate1_same_flag,paydate1_day,paydate1_weekday,paymentstartdate_weekday,lead_create_date_weekday,application_create_date_weekday,due_dt_weekday,paymentstartdate_paydate1_notsame_and_weekend_flag,…,employment_employer_unparsed,no_of_times_job_switched_12_months,no_of_times_job_switched_1_months,no_of_times_job_switched_24_months,no_of_times_job_switched_2_months,no_of_times_job_switched_36_months,no_of_times_job_switched_3_months,no_of_times_job_switched_48_months,no_of_times_job_switched_60_months,no_of_times_job_switched_6_months,no_of_times_job_switched_96_months,no_of_times_job_switched_ever,lti,income_rounding,payment_freq_bi_weekly,payment_freq_monthly,payment_freq_twice_per_month,payment_freq_weekly,payment_freq_other,repayment_freq_bi_weekly,repayment_freq_monthly,repayment_freq_twice_per_month,repayment_freq_weekly,employmentstatus_employment,employmentstatus_selfemployed,employmentstatus_governmentbenefit,employmentstatus_pension_or_annuity,employmentstatus_other,pti_1,pti_2,current_debt,current_usi_debt,cca_num_of_employers_last_six_months,cca_reported_net_monthly_income_previously_seen,RTI,DTI_current_debt,DTI_current_usi_debt
str,f64,str,datetime[ns],datetime[ns],datetime[ns],i32,str,datetime[ns],datetime[ns],f64,f64,f64,str,str,datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""LAI-00092480""",1.992233e6,"""546838288177@19H1""",2019-08-01 00:00:00,2019-09-01 00:00:00,2019-09-01 00:00:00,1,"""PS""",2019-08-15 00:00:00,2019-07-30 16:55:37,1.0,42000.0,2000.0,"""TWICE_PER_MONTH""","""TWICE_PER_MONTH""",2019-08-16 00:00:00,2019-07-31 08:59:13,1.0,NaN,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.0,15.0,3.0,4.0,1.0,2.0,6.0,0.0,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.025,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.029109,0.058217,787.0,NaN,1.0,5000.0,0.047619,0.018738,NaN
"""LAI-00121272""",2.636831e6,"""576839437287@19K1""",2019-10-25 00:00:00,2019-11-27 00:00:00,null,1,"""PS""",2019-10-30 00:00:00,2019-10-24 14:48:33,1.0,54600.0,3000.0,"""BI_WEEKLY""","""BI_WEEKLY""",2019-11-13 00:00:00,2019-10-24 14:51:11,4.0,NaN,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,30.0,2.0,2.0,3.0,3.0,2.0,0.0,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.028846,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.031782,0.068862,851.0,690.0,2.0,4500.0,0.054945,0.015586,0.012637
"""LAI-00201559""",4.471053e6,"""699961605241@21C1""",2021-03-02 00:00:00,2021-04-02 00:00:00,2021-04-02 00:00:00,1,"""PS""",2021-03-05 00:00:00,2021-03-01 14:01:00,1.0,26400.0,4000.0,"""BI_WEEKLY""","""BI_WEEKLY""",2021-03-19 00:00:00,2021-03-01 14:05:31,1.0,39999.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,5.0,4.0,4.0,0.0,0.0,4.0,0.0,…,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.059659,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0506,0.109633,308.0,NaN,0.0,2600.0,0.151515,0.011667,NaN
"""LAI-00215716""",5.127363e6,"""726939495270@21F2""",2021-06-15 00:00:00,2021-07-16 00:00:00,2021-07-16 00:00:00,1,"""PS""",2021-06-18 00:00:00,2021-06-15 12:32:33,1.0,39600.0,3000.0,"""BI_WEEKLY""","""BI_WEEKLY""",2021-07-02 00:00:00,2021-06-15 12:38:08,1.0,39999.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,28.0,18.0,4.0,4.0,1.0,1.0,4.0,0.0,…,NaN,1.0,0.0,1.0,0